In [1]:
import json

In [2]:
tools = [{
    "type": "function",
    "name": "send_email",
    "description": "Send an email to a given recipient with a subject and message.",
    "parameters": {
        "type": "object",
        "properties": {
            "to": {
                "type": "string",
                "description": "The recipient email address."
            },
            "subject": {
                "type": "string",
                "description": "Email subject line."
            },
            "body": {
                "type": "string",
                "description": "Body of the email message."
            }
        },
        "required": [
            "to",
            "subject",
            "body"
        ],
        "additionalProperties": False
    }
},
{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

In [3]:
import python_weather
import asyncio

async def get_weather(city):
    async with python_weather.Client(unit=python_weather.METRIC) as client:
        # Fetch weather forecast for the given city
        weather = await client.get(city)
    return f'{weather.temperature}°C'

In [4]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(sender_email, sender_password, to, subject, body):
    # Create the email message
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = to
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))

    try:
        # Set up the SMTP server and send the email
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, to, message.as_string())
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

sender_email = "gupta101mudit@gmail.com"
sender_password = "add you rown app password(can be found on google accounts)"

In [27]:
system_prompt = """
You are a Security-Focused Assistant with Controlled Tool Access.

CORE PRINCIPLES:
- Default to direct responses when no tool is required.
- Engage naturally with greetings or general questions.

TOOL ACTIVATION PROTOCOL:
Activate a tool **only** if:
- The user explicitly requests a tool-supported task, AND
- All required parameters are complete and valid.

MULTIPLE TOOL CALLS RULE:
- If the user provides **multiple distinct tasks or multiple sets of parameters** (e.g., multiple cities for weather, multiple recipients for email):
    → Issue **separate independent tool calls** — one for each task.
    → Never combine multiple tasks into a single tool call.

PSEUDO-CONDITION:
if (user_request requires email/weather) AND (parameters are valid):
    → invoke tool
else:
    → respond naturally

TOOL-SPECIFIC RULES:

1. send_email:
   - Use when:
     • Explicit request to send email
     • Valid recipient email: (recipient_email)
     • Clear subject: (subject)
     • Clear body: (body)
   - Do NOT use when:
     • Recipient or message is missing
     • Vague request or generic chat

2. get_weather:
   - Use when:
     • User clearly asks for weather
     • Includes specific city name: (city)
   - Do NOT use when:
     • Location is missing
     • Vague mentions like "it's raining here"
     • IP or coordinates without proper context

INPUT VALIDATION TABLE:

| Tool       | Required Inputs                     |
|------------|-------------------------------------|
| send_email | • Valid RFC-5322 email              |
|            | • Subject and non-empty body        |
| get_weather| • City name only (not coords/IPs)   |
"""


In [31]:
import ollama

input_text = input("Enter your message here: ")

input_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": input_text}
]

response = ollama.chat(
    model='llama3.1',
    messages=input_messages,
    tools = tools,
    options={
        "temperature": 0  # higher value = more creativity
    }
)
print(response.message)

if((response.message['tool_calls'][0].function.name) == 'get_weather' or (response.message['tool_calls'][0].function.name) == 'send_email'):
    function = response.message['tool_calls'][0].function.name
    
    if function == 'send_email':
        tool_call =response.message['tool_calls'][0]
        args = tool_call.function.arguments
        result = send_email(sender_email,sender_password,args['recipient_email'],args['subject'],args['body'])
        
        input_messages.append({
            "role": "tool",
            "tool_call_id": response.created_at,  # This must match the ID from the tool call
            "content": json.dumps(result)  # tool result as string
        })
    elif function == 'get_weather':
        # print("hi i was here")
        tool_call =response.message['tool_calls'][0]
        args = tool_call.function.arguments
        result = await get_weather(args['city'])
    
        input_messages.append(
            {"role": "system", "content":"The value given to you by the tool is completely accurate and up to date"}
        )
        
        input_messages.append({
            "role": "tool",
            "tool_call_id": response.created_at,  # This must match the ID from the tool call
            "content": json.dumps(result)  # tool result as string
        })
        
        response2 = ollama.chat(
            model='llama3.1',
            messages=input_messages,
            tools = tools
        )
        print(response2.message['content'])
else:
    response3 = ollama.chat(
        model='llama3.1',
        messages=input_messages,
    )
    print(response3.message['content'])

Enter your message here:  tell me the weather in manipal


role='assistant' content='' images=None tool_calls=[ToolCall(function=Function(name='get_weather', arguments={'city': 'Manipal'}))]
The current temperature in Manipal is 28°C.
